In [3]:
import os
import numpy as np
import zarr
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import PIL
import imageio

# load the configuration file
from l5kit.configs import load_config_data
from l5kit.configs.config import load_metadata
# load the dataset from zarr
from l5kit.data import ChunkedDataset, LocalDataManager
# load the Ego/Agent-based dataset
from l5kit.dataset import EgoDataset, AgentDataset, IntersectionDataset
# define the road_network protobuf data structure
from l5kit.data.proto.road_network_pb2 import RoadNetworkNode, RoadNetworkSegment
# Semantic map api
from l5kit.data.map_api import MapAPI
# rasterizer
from l5kit.rasterization import build_rasterizer
# geometry
from l5kit.geometry import transform_points
# trajectory
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR

In [4]:
# load the configuration file
cfg = load_config_data("./visualisation_config_train_1.yaml")
print(cfg)

# load the dataset from zarr
os.environ["L5KIT_DATA_FOLDER"] = "./" ## Set the environmental variable
dm = LocalDataManager()
dataset_path = dm.require(cfg["val_data_loader"]["key"])
# print(dataset_path) # scenes\sample.zarr
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()
print(zarr_dataset)

{'format_version': 4, 'model_params': {'model_architecture': 'resnet50', 'history_num_frames': 0, 'future_num_frames': 50, 'step_time': 0.1, 'render_ego_history': True}, 'raster_params': {'raster_size': [224, 224], 'pixel_size': [0.5, 0.5], 'ego_center': [0.25, 0.5], 'map_type': 'py_semantic', 'satellite_map_key': 'aerial_map/aerial_map.png', 'semantic_map_key': 'semantic_map/semantic_map.pb', 'dataset_meta_key': 'meta.json', 'filter_agents_threshold': 0.5, 'disable_traffic_light_faces': False, 'set_origin_to_bottom': True}, 'val_data_loader': {'key': 'scenes-train/train.zarr', 'batch_size': 12, 'shuffle': False, 'num_workers': 16}}
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+

In [5]:
# create the rasterizer and dataset

cfg["raster_params"]["map_type"] = "intersection" # "py_satellite" # "py_semantic"   
intersection_rast = build_rasterizer(cfg, dm)
intersection_dataset = IntersectionDataset(cfg, zarr_dataset, intersection_rast)

In [6]:
print(f"#frames in the {cfg['val_data_loader']['key']}: {len(intersection_dataset)}")
print(f"#scenes in the {cfg['val_data_loader']['key']}: {len(intersection_dataset.cumulative_sizes)}")

#frames in the scenes-train/train.zarr: 4039527
#scenes in the scenes-train/train.zarr: 16265


```python
# keys in a frame:

dict_keys(['frame_index', 'image', 'target_positions', 'target_yaws', 'target_velocities', 'target_availabilities', 
        'history_positions', 'history_yaws', 'history_velocities', 'history_availabilities', 
        'world_to_image', 'raster_from_agent', 'raster_from_world', 'agent_from_world', 'world_from_agent', 
        'centroid', 'yaw', 'extent', 'history_extents', 'future_extents', 'scene_index', 
        'host_id', 'timestamp', 'track_id', 'is_intersection_included'])
```

Iterate the dataset to get all the frames that include the intersection

In [7]:
frames_including_intersection = []
scenes_including_intersection = []

for idx, frame in tqdm(enumerate(intersection_dataset), 
                       total=len(intersection_dataset), 
                       unit="frame",
                       desc=f"Identify the intersection of interest inside {cfg['val_data_loader']['key']}:"):
    if frame["is_intersection_included"]:
        # save the frame_id
        frames_including_intersection.append(idx)
        # FIXME: save the scene_id
        scenes_including_intersection.append(frame["scene_index"])

Identify the intersection of interest inside scenes-train/train.zarr::   0%|          | 0/4039527 [00:00<?, ?f…

In [8]:
len(set(frames_including_intersection)), len(set(scenes_including_intersection))

(186992, 1290)

In [10]:
print(f"#scenes including the intersection:\n{len(set(scenes_including_intersection))}")

#scenes including the intersection:
1290


In [16]:
import pickle

# TODO: save the ids in picked files:
with open("./train1_scenes_including_intersection.pkl", 'wb') as file:
    pickle.dump(set(scenes_including_intersection), file)

with open("./train1_frames_including_intersection.pkl", 'wb') as file:
    pickle.dump(set(frames_including_intersection), file)

In [18]:
with open("./train1_scenes_including_intersection.pkl", 'rb') as file:
    set_scenes_including_intersection = pickle.load(file)

list_scenes_including_intersection = list(set_scenes_including_intersection)

with open("./train1_frames_including_intersection.pkl", 'rb') as file:
    set_frames_including_intersection = pickle.load(file)

list_frames_including_intersection = list(set_frames_including_intersection)

In [19]:
list_scenes_including_intersection2 = np.array(list_scenes_including_intersection[1:])
list_scenes_including_intersection1 = np.array(list_scenes_including_intersection[:-1])

diff = list_scenes_including_intersection2 - list_scenes_including_intersection1
np.sum(diff == 1)

671

## Visualization / Verification

Use GIF animation to check whether the frame includes the intersection

In [ ]:
from IPython.display import display, clear_output
import PIL
 
cfg["raster_params"]["map_type"] = "py_satellite"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)
scene_idx = 56
indexes = dataset.get_scene_indices(scene_idx)
images = [] # to store the frame for GIF

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"], data["raster_from_agent"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    # draw_trajectory(im, target_positions_pixels, TARGET_POINTS_COLOR, yaws=data["target_yaws"])
    clear_output(wait=True)
    display(PIL.Image.fromarray(im))

    pil_image = PIL.Image.fromarray(im)
    images.append(pil_image)

imageio.mimsave(f"./scene_gif/scene_{scene_idx}.gif", images)
clear_output(wait=True)

Use satellite map to check whether the frame includes the intersection

In [ ]:
frameIndex = 23120
cfg["raster_params"]["map_type"] = "py_satellite" # "intersection" # "py_semantic"   
rast = build_rasterizer(cfg, dm)
dataset = IntersectionDataset(cfg, zarr_dataset, rast)
data = dataset[frameIndex]

im = data["image"].transpose(1, 2, 0) # (224, 224, 5)
im = dataset.rasterizer.to_rgb(im) # (224, 224, 3)
target_positions_pixels = transform_points(data["target_positions"], data["raster_from_agent"])
# draw_trajectory(im, target_positions_pixels, TARGET_POINTS_COLOR, yaws=data["target_yaws"])

plt.imshow(im)
plt.show()